# Mislabeled data detection - RoBERTa-MRPC

This notebook demonstrates how to efficiently compute the influence functions using DataInf, showing its application to **mislabeled data detection** tasks.

- Model: Robert-large (https://arxiv.org/abs/1907.11692; pretrained with publicly available datasets including BOOKCORPUS, WIKIPEDIA, and CC-NEWS)
- Fine-tuning dataset: GLUE-mrpc
    - What is MRPC? The Microsoft Research Paraphrase Corpus (Dolan & Brockett, 2005) is a corpus of sentence pairs automatically extracted from online news sources, with human annotations for whether the sentences in the pair are semantically equivalent.

References
- https://github.com/huggingface/peft/blob/main/examples/sequence_classification/LoRA.ipynb
- DataInf is available at this [ArXiv link](https://arxiv.org/abs/2310.00902).

In [ ]:
from modelscope.msdatasets import MsDataset
# # Load the cola dataset
# train_datasets = MsDataset.load('glue', subset_name='cola', split='train')
# eval_datasets = MsDataset.load('glue', subset_name='cola', split='validation')

MsDataset.load(
            "glue",
            'qnli',
            cache_dir=None,
            use_auth_token=None,
        )

In [ ]:
import sys
sys.path.append('../src')
# from dataloader import create_dataloaders
from dataloader_ER import create_dataloaders
# from lora_model import LORAEngine,LORAEngineDeberta,LORAEngineDebertaMultiClass
from lora_model_vmap import LORAEngineDebertaMultiClass

# from influence import IFEngine
from influence_hyperinf import IFEngine

import numpy as np
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

## Set up hyperparameters and LoRA models

In [ ]:
# model_name_or_path="roberta-large"
# model_name_or_path="/home/yanmy/roberta-base"
model_name_or_path="../../model/deberta-v3-base"
# model_name_or_path="../../model/roberta-large"
# task="mrpc"
task = "ER"
noise_ratio=0
batch_size=32
# target_modules=["query_proj", "key_proj", "value_proj"] ## deberta, for roberta, use ["value"]
# target_modules = ['value']
target_modules=["value_proj"] ## deberta, for roberta, use ["value"]

device="cuda:3"
num_epochs=4
lr=3e-3

In [ ]:
# fine-tuning models
dataloader_outputs = create_dataloaders(model_name_or_path=model_name_or_path,
                                           task=task,
                                           noise_ratio=noise_ratio,
                                           batch_size=batch_size)
train_dataloader, eval_dataloader, noise_index, tokenized_datasets, collate_fn=dataloader_outputs

# lora_engine = LORAEngine(model_name_or_path=model_name_or_path,
#                             target_modules=target_modules,
#                             train_dataloader=train_dataloader,
#                             eval_dataloader=eval_dataloader,
#                             device=device,
#                             num_epochs=num_epochs,
#                             lr=lr,
#                             low_rank=8, 
#                             task=task)

lora_engine = LORAEngineDeberta(model_name_or_path=model_name_or_path,
                            target_modules=target_modules,
                            train_dataloader=train_dataloader,
                            eval_dataloader=eval_dataloader,
                            device=device,
                            num_epochs=num_epochs,
                            lr=lr,
                            low_rank=8, 
                            task=task)

## Fine-tune a model

In [ ]:
# fine-tuning models
# dataloader_outputs = create_dataloaders(model_name_or_path=model_name_or_path,
#                                            task=task,
#                                            noise_ratio=noise_ratio,
#                                            batch_size=batch_size,
#                                            train_file = '../ER/semi-text-c-FUSER/train.json',
#                                            valid_file = '../ER/semi-text-c-FUSER/valid.json',
#                                            test_file = '../ER/semi-text-c-FUSER/test.json')


dataloader_outputs = create_dataloaders(model_name_or_path=model_name_or_path,
                                           task=task,
                                           noise_ratio=noise_ratio,
                                           batch_size=batch_size,
                                           train_file = '../ER/semi-text-c/train.json',
                                           valid_file = '../ER/semi-text-c/valid.json',
                                           test_file = '../ER/semi-text-c/test.json',
                                           max_length=256)
# train_dataloader, eval_dataloader, noise_index, tokenized_datasets, collate_fn=dataloader_outputs
train_dataloader, eval_dataloader,test_dataloader, noise_index, tokenized_datasets, collate_fn=dataloader_outputs


# lora_engine = LORAEngine(model_name_or_path=model_name_or_path,
#                             target_modules=target_modules,
#                             train_dataloader=train_dataloader,
#                             eval_dataloader=eval_dataloader,
#                             device=device,
#                             num_epochs=num_epochs,
#                             lr=lr,
#                             low_rank=8, 
#                             task=task)

lora_engine = LORAEngineDebertaMultiClass(model_name_or_path=model_name_or_path,
                            target_modules=target_modules,
                            train_dataloader=train_dataloader,
                            # eval_dataloader=eval_dataloader,
                            eval_dataloader=eval_dataloader,
                            test_dataloader = test_dataloader,
                            device=device,
                            num_epochs=num_epochs,
                            lr=lr,
                            low_rank=16, 
                            task=task,
                            save_path = '../models/ER',
                            valid_each_epoch=False)

In [ ]:
lora_engine.build_LORA_model()
lora_engine.train_LORA_model()

## Compute the gradient
 - Influence function uses the first-order gradient of a loss function. Here we compute gradients using `compute_gradient`
 - `tr_grad_dict` has a nested structure of two Python dictionaries. The outer dictionary has `{an index of the training data: a dictionary of gradients}` and the inner dictionary has `{layer name: gradients}`. The `val_grad_dict` has the same structure but for the validationd data points. 

## Compute the influence function
 - We compute the inverse Hessian vector product first using `compute_hvps()`. With the argument `compute_accurate=True`, the exact influence function value will be computed. (it may take an hour to compute).

In [ ]:
tr_grad_dict, val_grad_dict = lora_engine.compute_gradient(tokenized_datasets, collate_fn, batch_size=32)

In [ ]:
from time import time
from tqdm.notebook import tqdm
import sys
from collections import defaultdict
import pandas as pd
import pickle, os
import torch
import numpy as np
class IFEngine(object):
        # 初始化存储时间、Hessian-Vector Product (HVP) 结果和影响函数（Influence Function）值的字典

    def __init__(self):
        self.time_dict=defaultdict(list)
        self.hvp_dict=defaultdict(list)
        self.IF_dict=defaultdict(list)

    def preprocess_gradients(self, tr_grad_dict, val_grad_dict, noise_index=None): # 存储训练集和验证集的梯度字典，并计算验证集平均梯度
        self.tr_grad_dict = tr_grad_dict # 训练集梯度字典
        self.val_grad_dict = val_grad_dict # 验证集梯度字典
        self.noise_index = noise_index # 可选的噪声索引（用于异常值处理）
        # 存储训练集和验证集的样本数
        self.n_train = len(self.tr_grad_dict.keys())
        self.n_val = len(self.val_grad_dict.keys())
        self.compute_val_grad_avg() # 计算验证集平均梯度

    def compute_val_grad_avg(self):
        # Compute the avg gradient on the validation dataset
        self.val_grad_avg_dict={}
        for weight_name in self.val_grad_dict[0]:
            # 初始化为与梯度同设备的零向量
            self.val_grad_avg_dict[weight_name]=torch.zeros(self.val_grad_dict[0][weight_name].shape).to(self.val_grad_dict[0][weight_name].device)
            # 逐个样本累加梯度并取平均值
            for val_id in self.val_grad_dict:
                self.val_grad_avg_dict[weight_name] += self.val_grad_dict[val_id][weight_name] / self.n_val

    def compute_hvps(self, lambda_const_param=10, compute_accurate=True, compute_LiSSA=True):
        '''
        Compute the influence function score under each method
        '''
        self.compute_hvp_iterative(lambda_const_param=lambda_const_param) ## HyperInf,使用 Schulz 迭代法计算 HVP 和近似 Hessian 逆矩阵
        self.compute_hvp_identity() ## Baseline TracIn
        self.compute_hvp_proposed(lambda_const_param=lambda_const_param) ## Datainf
        if compute_LiSSA:
            self.compute_hvp_LiSSA(lambda_const_param=lambda_const_param)
        if compute_accurate:
            self.compute_hvp_accurate(lambda_const_param=lambda_const_param)

    def compute_hvp_identity(self):
        '''
        TracIN
        '''
        start_time = time()
        self.hvp_dict['identity'] = self.val_grad_avg_dict.copy()
        self.time_dict['identity'] = time()-start_time
        print("Time taken for Hessian-free: ", self.time_dict['identity'])

    def compute_hvp_iterative(self, lambda_const_param=10, n_iteration=30):
        '''
        Compute the influence funcion score by our method HyperINF
        '''

        def schulz_inverse_stable(A, damping_factor=0, max_iterations=20, tol=1e-6):
            n = A.shape[0]
            #I = np.eye(n)
            I = torch.eye(n, device=A.device)
            A_damped = A + damping_factor * I  # Apply damping for stable

            #X = np.eye(n) * 0.00005  # Initial estimate of inverse matrix
            X = torch.eye(n, device=A.device) * 0.00005  # Initial estimate of inverse matrix

            for _ in range(max_iterations):
                # 通过 Schulz 方法更新逆矩阵估计值
                #X = X.dot(2 * I - A_damped.dot(X))
                X = X @ (2 * I - A_damped @ X)

                # # Check for convergence
                # if np.linalg.norm(I - A.dot(X)) < tol:
                #     break

            return X

        start_time = time()
        hvp_iterative_dict={}

        for _, weight_name in enumerate(tqdm(self.val_grad_avg_dict)):
            # lambda_const computation = 0.1 x (n * d_l)^(-1) \sum_{i=1}^{n} ||grad_i^l||_2^2, 计算 λ（正则化系数）：基于梯度的方差
            S=torch.zeros(len(self.tr_grad_dict.keys())).to(self.val_grad_avg_dict[weight_name].device)
            for tr_id in self.tr_grad_dict:
                tmp_grad = self.tr_grad_dict[tr_id][weight_name]
                S[tr_id]=torch.mean(tmp_grad**2)
            lambda_const = torch.mean(S) / lambda_const_param # layer-wise lambda

            # iterative hvp computation
            # G_l: same shape of self.tr_grad_dict[0][weight_name].T @ self.tr_grad_dict[0][weight_name]
            # 构造 Fisher 信息矩阵（近似 Hessian）
            G_l = torch.zeros((self.tr_grad_dict[0][weight_name].T @ self.tr_grad_dict[0][weight_name]).shape).to(self.val_grad_avg_dict[weight_name].device)
            for tr_id in self.tr_grad_dict:
                tmp_grad = self.tr_grad_dict[tr_id][weight_name].to(self.val_grad_avg_dict[weight_name].device) # (grad_i^l)^T
                G_l += tmp_grad.T @ tmp_grad / self.n_train
                
            # 加入正则化项以保证矩阵可逆
            G_l = G_l + lambda_const * torch.eye(G_l.shape[0], device=G_l.device)
           # G_l = G_l.cpu().detach().numpy()
           # 使用 Schulz 方法近似计算逆矩阵
            # G_l_inv = schulz_inverse_stable(G_l, damping_factor=0.001, max_iterations=n_iteration, tol=1e-6)
            G_l_inv = torch.inverse(G_l)
            print(G_l.shape,G_l_inv.shape)
            # 计算 HVP: G_l_inv @ val_grad_avg
            hvp_iterative_dict[weight_name] = torch.tensor(self.val_grad_avg_dict[weight_name] @ G_l_inv)
            #print(hvp_iterative_dict[weight_name])
        self.hvp_dict['iterative'] = hvp_iterative_dict
        self.time_dict['iterative'] = time()-start_time
        print("Time taken for HyperINF: ", self.time_dict['iterative'])



    def compute_hvp_proposed(self, lambda_const_param=10):
        '''
        DataInf method
        '''
        start_time = time()
        hvp_proposed_dict={}

        for _ , weight_name in enumerate(tqdm(self.val_grad_avg_dict)):
            # lambda_const computation = 0.1 x (n * d_l)^(-1) \sum_{i=1}^{n} ||grad_i^l||_2^2
            S=torch.zeros(len(self.tr_grad_dict.keys())).to(self.val_grad_avg_dict[weight_name].device)
            for tr_id in self.tr_grad_dict:
                tmp_grad = self.tr_grad_dict[tr_id][weight_name].to(self.val_grad_avg_dict[weight_name].device)
                S[tr_id]=torch.mean(tmp_grad**2)
            lambda_const = torch.mean(S) / lambda_const_param # layer-wise lambda

            # hvp computation
            hvp=torch.zeros(self.val_grad_avg_dict[weight_name].shape).to(self.val_grad_avg_dict[weight_name].device)
            for tr_id in self.tr_grad_dict: # i
                tmp_grad = self.tr_grad_dict[tr_id][weight_name].to(self.val_grad_avg_dict[weight_name].device) # grad_i^l
                # L_(l,i) / (lambda + ||grad_i^l||_2^2) in Eqn. (5)
                C_tmp = torch.sum(self.val_grad_avg_dict[weight_name] * tmp_grad) / (lambda_const + torch.sum(tmp_grad**2)).to(self.val_grad_avg_dict[weight_name].device)
                # (v_l^T - C_tmp * (grad_i^l)^T ) / (n * lambda) in Eqn. (5)
                hvp += (self.val_grad_avg_dict[weight_name] - C_tmp*tmp_grad) / (self.n_train*lambda_const)
            hvp_proposed_dict[weight_name] = hvp
        self.hvp_dict['proposed'] = hvp_proposed_dict
        self.time_dict['proposed'] = time()-start_time
        print("Time taken for Datainf: ", self.time_dict['proposed'])

    def compute_hvp_accurate(self, lambda_const_param=10):
        start_time = time()
        hvp_accurate_dict={}
        for _ , weight_name in enumerate(tqdm(self.val_grad_avg_dict)):

            # lambda_const computation
            S=torch.zeros(len(self.tr_grad_dict.keys()))
            for tr_id in self.tr_grad_dict:
                tmp_grad = self.tr_grad_dict[tr_id][weight_name]
                S[tr_id]=torch.mean(tmp_grad**2)
            lambda_const = torch.mean(S) / lambda_const_param # layer-wise lambda

            # hvp computation (eigenvalue decomposition)
            AAt_matrix = torch.zeros(torch.outer(self.tr_grad_dict[0][weight_name].reshape(-1),
                                                 self.tr_grad_dict[0][weight_name].reshape(-1)).shape)
            for tr_id in self.tr_grad_dict:

                tmp_mat = torch.outer(self.tr_grad_dict[tr_id][weight_name].reshape(-1),
                                      self.tr_grad_dict[tr_id][weight_name].reshape(-1))
                AAt_matrix += tmp_mat


            L, V = torch.linalg.eig(AAt_matrix)
            L, V = L.float(), V.float()
            hvp = self.val_grad_avg_dict[weight_name].reshape(-1) @ V
            hvp = (hvp / (lambda_const + L/ self.n_train)) @ V.T

            hvp_accurate_dict[weight_name] = hvp.reshape(len(self.tr_grad_dict[0][weight_name]), -1)
            del tmp_mat, AAt_matrix, V # to save memory
        self.hvp_dict['accurate'] = hvp_accurate_dict
        self.time_dict['accurate'] = time()-start_time
        print("Time taken for Accurate: ", self.time_dict['accurate'])

    def compute_hvp_LiSSA(self, lambda_const_param=10, n_iteration=10, alpha_const=1.):
        '''
        LiSSA method
        '''
        start_time = time()
        hvp_LiSSA_dict={}

        for _, weight_name in enumerate(tqdm(self.val_grad_avg_dict)):
            # lambda_const computation
            S=torch.zeros(len(self.tr_grad_dict.keys())).to(self.val_grad_avg_dict[weight_name].device)
            for tr_id in self.tr_grad_dict:
                tmp_grad = self.tr_grad_dict[tr_id][weight_name].to(self.val_grad_avg_dict[weight_name].device)
                S[tr_id]=torch.mean(tmp_grad**2)
            lambda_const = torch.mean(S) / lambda_const_param # layer-wise lambda

            # hvp computation
            running_hvp=self.val_grad_avg_dict[weight_name]
            for _ in range(n_iteration):
                hvp_tmp=torch.zeros(self.val_grad_avg_dict[weight_name].shape).to(self.val_grad_avg_dict[weight_name].device)
                for tr_id in self.tr_grad_dict:
                    tmp_grad = self.tr_grad_dict[tr_id][weight_name].to(self.val_grad_avg_dict[weight_name].device)
                    hvp_tmp += (torch.sum(tmp_grad*running_hvp)*tmp_grad - lambda_const*running_hvp) / self.n_train

                running_hvp = self.val_grad_avg_dict[weight_name] + running_hvp - alpha_const*hvp_tmp
            hvp_LiSSA_dict[weight_name] = running_hvp

        self.hvp_dict['LiSSA'] = hvp_LiSSA_dict
        self.time_dict['LiSSA'] = time()-start_time
        print("Time taken for LiSSA: ", self.time_dict['LiSSA'])

    def compute_IF(self):
        for method_name in self.hvp_dict:
            if_tmp_dict = {}
            for tr_id in self.tr_grad_dict:
                if_tmp_value = 0
                for weight_name in self.val_grad_avg_dict:
                    if_tmp_value += torch.sum(self.hvp_dict[method_name][weight_name]*self.tr_grad_dict[tr_id][weight_name])
                if_tmp_dict[tr_id]= -if_tmp_value.cpu()
               # print(-if_tmp_value)

            self.IF_dict[method_name] = pd.Series(if_tmp_dict, dtype=float).to_numpy()


In [ ]:
influence_engine = IFEngine()
influence_engine.preprocess_gradients(tr_grad_dict, val_grad_dict, noise_index)

influence_engine.compute_hvps(compute_accurate=False,compute_LiSSA=False)
influence_engine.compute_IF()

In [ ]:
influence_engine.IF_dict['proposed']

In [ ]:
# result = torch.load('/home/yanmy/HyperINF-main/output/output_fp16_deberta_ER_v_proj.pkl')
result['proposed']

In [ ]:
from tqdm.notebook import tqdm
IF_device = 'cuda:3' ## 避免爆显存
for key in tqdm(tr_grad_dict): ## 尝试转为fp16计算
    for kk in tr_grad_dict[key]:
        tr_grad_dict[key][kk] = tr_grad_dict[key][kk].to(IF_device)
for key in tqdm(val_grad_dict):
    for kk in val_grad_dict[key]:
        val_grad_dict[key][kk] = val_grad_dict[key][kk].to(IF_device)

influence_engine = IFEngine()
influence_engine.preprocess_gradients(tr_grad_dict, val_grad_dict, noise_index)

influence_engine.compute_hvps(compute_accurate=False,compute_LiSSA=True)
influence_engine.compute_IF()

In [ ]:
torch.save(influence_engine.IF_dict,'../../HyperINF-main/output/output_fp16_deberta_ER_v_proj_inv.pkl')

In [1]:
import torch
from tqdm.notebook import tqdm
tr_grad_dict = torch.load('../output/tr_grad_dict_all.pkl')
val_grad_dict = torch.load('../output/val_grad_dict_all.pkl')

/tmp/ipykernel_3288995/2153823836.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tr_grad_dict = torch.load('../output/tr_grad_dict_all.pkl')
/tmp/ipykernel_3288995/2153

In [2]:
tr_grad_dict = tr_grad_dict[0]
val_grad_dict = val_grad_dict[0]
# tr_grad_dict.pop('ids')

In [ ]:
for key in tqdm(val_grad_dict):
    val_grad_dict[key].pop('ids')
for key in tqdm(tr_grad_dict):
    tr_grad_dict[key].pop('ids')

In [3]:
import sys
sys.path.append('../src')
from influence_batch import IFEngine
from tqdm.notebook import tqdm

IF_device = 'cuda:7' ## 避免爆显存
# tr_grad_dict_IF = tr_grad_dict.copy()
# val_grad_dict_IF = val_grad_dict.copy()
# ## to cuda
for key in tqdm(tr_grad_dict): ## 尝试转为fp16计算
    for kk in tr_grad_dict[key]:
        # if kk!='ids':
            tr_grad_dict[key][kk] = tr_grad_dict[key][kk].to(IF_device)
        # tr_grad_dict[key].pop('ids')

for key in tqdm(val_grad_dict):
    for kk in val_grad_dict[key]:
        # if kk!='ids':
            val_grad_dict[key][kk] = val_grad_dict[key][kk].to(IF_device)
        # val_grad_dict[key].pop('ids')


noise_index = None
## compute influence function


  0%|          | 0/115 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [8]:
influence_engine = IFEngine(weight_list=list(val_grad_dict[0].keys())[22:])
influence_engine.preprocess_gradients(tr_grad_dict, val_grad_dict, noise_index)

influence_engine.compute_hvps(compute_accurate=False,compute_LiSSA=True)
influence_engine.compute_IF()

  0%|          | 0/4 [00:00<?, ?it/s]

Time taken for HyperINF:  0.09330606460571289
Time taken for Hessian-free:  1.6689300537109375e-06


/data/home/wangys/DataSelection-IF/notebooks/../src/influence_batch.py:114: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  hvp_iterative_dict[weight_name] = torch.tensor(self.val_grad_avg_dict[weight_name] @ G_l_inv)


  0%|          | 0/4 [00:00<?, ?it/s]

Time taken for Datainf:  0.06619572639465332


  0%|          | 0/4 [00:00<?, ?it/s]

Time taken for LiSSA:  0.24889206886291504


In [10]:
result = influence_engine.IF_dict['proposed']
import pandas as pd
pd.DataFrame(result)

,0,1,2,3,4,5,6,7,8,9,...,105,106,107,108,109,110,111,112,113,114
base_model.model.deberta.encoder.layer.10.attention.self.value_proj.lora_B.default.weight,4246.681152,-32049.476562,5386.741699,4397.683105,-4365.251953,-28306.96875,5716.95166,-24101.621094,-24783.480469,8573.176758,...,-10405.790039,8494.412109,-12377.030273,-4557.339355,-13784.908203,9479.004883,5446.173828,-13826.923828,708.128052,-10125.771484
base_model.model.deberta.encoder.layer.11.attention.self.value_proj.lora_A.default.weight,4468.546387,-28445.125,2920.133789,8445.088867,-86.586975,-39438.757812,8688.3125,-23035.714844,-23578.392578,8994.666016,...,-14895.918945,13628.555664,-13053.261719,-3069.782227,-2677.004883,12798.408203,20255.876953,-18925.603516,5608.026367,-14934.188477
base_model.model.deberta.encoder.layer.11.attention.self.value_proj.lora_B.default.weight,-18412.275391,-17422.791016,5109.820312,5936.246094,-6635.799805,-19055.220703,1715.612061,-21720.251953,-6955.642578,-480.379883,...,-8185.606934,5542.118652,-9518.250977,-21220.15625,-30517.861328,11464.677734,8343.212891,-4946.064941,10544.654297,-3580.837891
base_model.model.classifier.modules_to_save.default.weight,1828.411377,-1521.543823,-438.234985,30.565327,90.412445,-1516.199341,250.946869,-874.574341,-1554.590332,985.943359,...,-898.345337,1036.105713,-694.516724,1367.785645,1204.49292,739.494812,1580.711304,-1676.055054,285.42157,-1256.068359
ids,"[2041, 3032, 496, 1522, 2068, 2994, 1911, 535,...","[1189, 2592, 146, 588, 1106, 3043, 2499, 1623,...","[1943, 3503, 2542, 1033, 469, 951, 243, 3315, ...","[802, 2966, 2274, 99, 557, 2622, 3121, 349, 47...","[630, 1832, 75, 3001, 2569, 3508, 407, 1796, 1...","[1784, 650, 1013, 485, 2294, 1403, 2546, 2885,...","[2269, 1569, 2621, 589, 2466, 498, 165, 2962, ...","[44, 978, 548, 778, 492, 122, 3657, 3603, 2243...","[411, 1417, 2456, 136, 2566, 191, 1621, 1315, ...","[1161, 2547, 3322, 1420, 2909, 2362, 3098, 994...",...,"[1908, 1060, 747, 184, 1059, 3218, 2916, 1765,...","[2743, 2624, 1310, 1743, 2616, 1986, 3325, 321...","[347, 3115, 2064, 910, 1731, 766, 1174, 751, 3...","[659, 3630, 23, 3019, 5, 1857, 3000, 1709, 297...","[3078, 1950, 2646, 1828, 1179, 2989, 754, 607,...","[807, 1239, 187, 1864, 708, 281, 1815, 912, 25...","[2048, 1020, 3419, 3548, 2307, 1198, 3666, 374...","[2839, 42, 1474, 3438, 2714, 1782, 1800, 2654,...","[3169, 3622, 2114, 1759, 897, 1391, 2783, 1845...","[3093, 272, 1783, 58, 728, 2075, 820, 1912, 15..."


,0,1,2,3,4,5,6,7,8,9,...,105,106,107,108,109,110,111,112,113,114
base_model.model.deberta.encoder.layer.0.attention.self.value_proj.lora_A.default.weight,-0.002792,0.214461,-0.137952,0.092945,-0.113937,-0.248649,0.377446,-0.55323,-0.42092,-0.162539,...,-0.256942,-0.060108,-0.30166,-0.587831,-0.019633,-0.041786,-0.036602,0.03769,0.091819,0.160528
base_model.model.deberta.encoder.layer.0.attention.self.value_proj.lora_B.default.weight,0.056509,-0.156315,-0.140347,0.101002,-0.337351,-0.438037,0.197004,-0.320979,-0.644424,-0.0436,...,-0.297538,-0.254762,-0.707609,-0.628391,-0.296118,-0.033697,-0.340017,0.095167,0.067727,0.054598
base_model.model.deberta.encoder.layer.1.attention.self.value_proj.lora_A.default.weight,0.178758,-0.66987,0.163777,0.060063,-0.190005,-0.300544,0.020764,-0.143777,-0.676488,0.024625,...,-0.291554,0.210634,-0.045833,-0.283357,-0.402362,-0.40779,-0.39959,0.281333,0.205976,0.327814
base_model.model.deberta.encoder.layer.1.attention.self.value_proj.lora_B.default.weight,0.027288,0.010155,0.038804,-0.003856,-0.204447,-0.210844,-0.103285,-0.035615,-0.329397,-0.14137,...,-0.157061,-0.352993,-0.375491,-0.329159,-0.029132,-0.038246,-0.058219,-0.000214,0.007177,-0.036041
base_model.model.deberta.encoder.layer.2.attention.self.value_proj.lora_A.default.weight,0.055865,-0.081093,-0.01777,-0.029478,-0.237979,-0.286873,0.074688,-0.318219,-0.379533,0.151725,...,-0.264797,0.023683,-0.265076,-0.063408,-0.145172,-0.09048,-0.029833,0.017699,-0.004159,0.069041
base_model.model.deberta.encoder.layer.2.attention.self.value_proj.lora_B.default.weight,0.14476,-0.074117,0.040006,-0.003116,-0.344114,-0.356342,0.184545,-0.23669,-0.592648,-0.005764,...,-0.277656,-0.132468,-0.196689,-0.199692,-0.262067,-0.134191,-0.168306,-0.028317,-0.111756,-0.037405
base_model.model.deberta.encoder.layer.3.attention.self.value_proj.lora_A.default.weight,0.136803,-0.192393,0.091636,-0.044098,-0.244484,-0.471105,0.359509,-0.169028,-0.858913,-0.062963,...,-0.473988,-0.036099,-0.369804,-0.006071,-0.265601,-0.087226,-0.213986,-0.179511,0.010667,0.200177
base_model.model.deberta.encoder.layer.3.attention.self.value_proj.lora_B.default.weight,0.139233,-0.073114,0.120961,-0.123164,-0.346416,-0.051335,0.070784,-0.128176,-0.906164,-0.005602,...,-0.260905,-0.276575,-0.198805,-0.185212,-0.193549,-0.247312,-0.362922,-0.083613,0.069094,0.139744
base_model.model.deberta.encoder.layer.4.attention.self.value_proj.lora_A.default.weight,0.19205,-0.590967,0.032571,-0.005729,-0.235471,-0.84766,0.188607,-0.500709,-1.083541,-0.238263,...,-0.329472,-0.047762,-0.617092,-0.29318,-0.246804,-0.166606,-0.106733,-0.248479,0.15717,-0.134625
base_model.model.deberta.encoder.layer.4.attention.self.value_proj.lora_B.default.weight,0.392455,-0.44349,0.188913,0.018775,-0.391593,-0.448656,0.582309,0.05325,-0.966565,0.250896,...,-0.244405,0.020122,-0.476967,-0.314586,-0.023449,-0.238637,-0.594619,-0.246472,0.107189,-0.046298


## Attributes of influence_engine
There are a couple of useful attributes in `influence_engine`. For intance, to compare the runtime, one case use `time_dict`.

`IF_dict` includes all the computed influence function values. Here, `identity` indicates the `Hessian-free` influence computation method. 

In [ ]:
# -*- encoding:utf-8 -*-
import torch
import numpy as np
import transformers
print(torch.__version__)  # 1.7.1
print(transformers.__version__) # 2.1.1

In [ ]:
from transformers import (
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup,
    BitsAndBytesConfig,
    LlamaForCausalLM,
    LlamaTokenizer,
    DebertaV2ForSequenceClassification, DebertaV2Tokenizer, AdamW, get_linear_schedule_with_warmup,AutoTokenizer
)
tokenizer = AutoTokenizer.from_pretrained('/home/yanmy/model/deberta-v3-base/')

In [ ]:
import torch
torch.save(influence_engine.IF_dict,'../../HyperINF-main/output/output_fp16_deberta_all.pkl')

In [ ]:
influence_engine.IF_dict['identity']

## retrain

In [ ]:
import torch
import numpy as np
result = torch.load('/home/yanmy/HyperINF-main/output/output_fp16_deberta.pkl')
index = np.argsort(result['proposed'])[500:1000]

In [ ]:
import pandas as pd
pd.read_json('../ER/semi-text-c/train.json').iloc[index,-1].value_counts()

In [ ]:
a = np.argsort(result['proposed'])[:1000]

In [ ]:
result = torch.load('/home/yanmy/HyperINF-main/output/output_deberta_fp16_ER.pkl')
b = np.argsort(result['proposed'])[:1000]

In [ ]:
len(set(a).intersection(set(b)))

In [ ]:
import torch
import numpy as np
result = torch.load('/home/yanmy/HyperINF-main/output/output_fp16_deberta_ER_v_proj_inv.pkl')
index = np.argsort(result['iterative'])[:1500]
import pandas as pd
pd.read_json('../ER/semi-text-c/train.json').iloc[index,-1].value_counts()

In [ ]:
model_name_or_path="/home/yanmy/model/deberta-v3-base" 
# task="mrpc"
task = "ER"
noise_ratio=0
batch_size=32
# target_modules=["query_proj", "key_proj", "value_proj"] ## deberta, for roberta, use ["value"]
target_modules=["value_proj"] ## deberta, for roberta, use ["value"]

device="cuda:3"
num_epochs=10
lr=3e-4


dataloader_outputs = create_dataloaders(model_name_or_path=model_name_or_path,
                                           task=task,
                                           noise_ratio=noise_ratio,
                                           batch_size=batch_size,
                                           train_file = '../ER/semi-text-c/train.json',
                                           valid_file = '../ER/semi-text-c/valid.json',
                                           test_file = '../ER/semi-text-c/test.json',
                                           max_length=256,
                                           select_index=index)
# train_dataloader, eval_dataloader, noise_index, tokenized_datasets, collate_fn=dataloader_outputs
train_dataloader, eval_dataloader,test_dataloader, noise_index, tokenized_datasets, collate_fn=dataloader_outputs


# lora_engine = LORAEngine(model_name_or_path=model_name_or_path,
#                             target_modules=target_modules,
#                             train_dataloader=train_dataloader,
#                             eval_dataloader=eval_dataloader,
#                             device=device,
#                             num_epochs=num_epochs,
#                             lr=lr,
#                             low_rank=8, 
#                             task=task)

lora_engine = LORAEngineDebertaMultiClass(model_name_or_path=model_name_or_path,
                            target_modules=target_modules,
                            train_dataloader=train_dataloader,
                            # eval_dataloader=eval_dataloader,
                            eval_dataloader=eval_dataloader,
                            test_dataloader = test_dataloader,
                            device=device,
                            num_epochs=num_epochs,
                            lr=lr,
                            low_rank=8, 
                            task=task,
                            save_path = '../models/ER',
                            valid_each_epoch = True
                            )

`output/output_fp16_deberta.pkl`
- 1000: {'f1': 0.7783063748810657}

## Application to mislabeled data detection task
- We compare the mislabeled data detection ability of different influence computation methods. Given that large influence function values are likely to increase the validation loss, data points with large influence fucntion values are desired to be mislabeled. 
- We inspect data points from the largest to lowest influence function values and evaluate the detection rate. 

In [ ]:
n_train=influence_engine.n_train
true_label=np.zeros(n_train)
true_label[noise_index]=1

method_dict={'identity': 'Hessian-free',
            'proposed': 'DataInf',
            'LiSSA': 'LiSSA'}

Requirement already satisfied: torch in /home/yanmy/anaconda3/lib/python3.9/site-packages (1.13.1)

In [ ]:
plt.figure(figsize=(5,4))
for method in influence_engine.IF_dict:
    detection_rate_list=[]
    low_quality_to_high_quality=np.argsort(influence_engine.IF_dict[method])[::-1]
    for ind in range(1, len(low_quality_to_high_quality)+1):
        detected_samples = set(low_quality_to_high_quality[:ind]).intersection(noise_index)
        detection_rate = 100*len(detected_samples)/len(noise_index)
        detection_rate_list.append(detection_rate)
    plt.plot(100*np.arange(len(low_quality_to_high_quality))/n_train, 
             detection_rate_list,
             label=method_dict[method])
plt.xlabel('Data inspected (%)', fontsize=18)
plt.ylabel('Detection Rate (%)', fontsize=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=15)
plt.title('Mislabeled Data Detection', fontsize=15)
plt.show()